 # Least Squares
In diesem Notebook werden wir lineare Regression (least squares) implementieren und auf die Auto-Daten anwenden, um Gebrauchtwagenpreise auf EBay vorherzusagen.

## 1. Implementierung

Implementieren Sie eine Methode *leastsquares()*, die gemäß der Formel aus den Videos die Parameter $(a,b)$ einer Ausgleichsgerade ermittelt und zurückliefert.
Ihre Methode erhält mit x und y zwei Numpy-Arrays der Länge $n$ mit den x- und y-Koordinaten der gegebenen Stichprobe.

*Hinweis: Sie benötigen Kennwerte wie Mittelwert oder Varianz. Verwenden Sie hierzu numpy-Methoden wie mean() und var().*

*Hinweis: Sie können zur Berechnung der Kovarianz die Methode np.cov() heranziehen. Falls Sie dies möchten, setzen Sie den Parameter bias auf True.*


In [1]:
import numpy as np

def leastsquares(x,y):
    arr2D = np.asarray([x, y])#np.vstack([x, y]).T
    avg_x = np.mean(x)
    avg_y = np.mean(y)
    
    Sxy = np.cov(arr2D, bias=True)[0][1]
    Sx2 = np.var(x)
    alpha = Sxy / Sx2
    beta = avg_y - alpha * avg_x
    
    #return np.dot((np.dot(np.linalg.inv(np.dot(x.T,x)),x.T)),y[:, np.newaxis])
    return [alpha, beta]

## 2. Kleiner Test
Wir verwenden zunächst eine kleine Stichprobe: 
$$(x_1,y_1), ..., (x_4,y_4) = (1,1), (2,2), (3,5), (4,7).
$$
Testen Sie Ihre Methode *leastsquares()*, indem Sie sie auf die Daten anwenden und dann Daten und Gerade mit der Funktion *plot()* plotten.

In [2]:
import matplotlib.pyplot as plt

def plot(x, y, a, b):
    """ 
          ! DO NOT TOUCH !

          This method plots the 2D point cloud (x,y) in red, 
          and the fitted line (a,b) in orange.
    """
    plt.plot(x, y, 'o', color='red', markersize=5)
    xmin = np.min(x)-1
    xmax = np.max(x)+1
    ymin = np.min(y)-1
    ymax = np.max(y)+1    
    plt.axis([xmin,xmax,ymin,ymax]) # set range to plot
    xticks = np.arange(xmin, xmax, (xmax-xmin)/1000.)
    plt.plot(xticks, a*xticks+b, '-', color='orange')
    
    plt.grid(True)
    plt.show()    
    
x = np.array([1,2,3,4])
y = np.array([1,2,5,7])
a, b = leastsquares(x,y)
plot(x,y,a,b)

## 3. Autodaten laden
Laden Sie die bekannten Autodaten 'cars.csv'. Führen Sie hierzu die nächste Zelle aus.

In [3]:
import pandas

# Anmerkung: Man kann die Datei auch in Excel/Libreoffice öffnen.
cars = pandas.read_csv('/kaggle/input/carscsv/cars.csv')

# Wir geben die Namen der Spalten/Merkmale aus.
columns = cars.columns
for i,col in enumerate(columns):
    print('%.2d : %s' %(i,col))
    
# Wir konvertieren die Daten in ein Numpy-Array.
data = cars.values.astype('float')

# Wir filtern alle Autos die teurer als 50.000 EUR sind.
data = data[data[:,0]<50000,:]
# Wir filtern alle Autos die billiger als 1.000 EUR sind.
data = data[data[:,0]>=1000,:]

## 4. Autopreise vorhersagen
Verwenden Sie nun Ihr Least-Squares-Modell, um den *Preis* eines Autos vorherzusagen, gegeben den *Kilometerstand*. 
* Extrahieren Sie hierzu beide Spalten aus der Matrix *data* und rufen Sie leastsquares() auf, um die Parameter $a$ und $b$ zu bestimmen.
* Ermitteln Sie dann für die ersten zehn Autos die Vorhersage Ihres Modells und vergleichen Sie sie mit den echten Preisen.

Wirken die Ergebnisse (halbwegs) plausibel?

In [4]:
def calculatePrice(alpha, beta, price):
    return alpha * price + beta

a = data[:, 0] #prices
b = data[:, 3] #km

res = leastsquares(a, b)

for i in range(10):
    print(a[i], " : ", b[i], " -> ", calculatePrice(res[0], res[1], a[i]))

## 5. Modell bewerten
Schreiben Sie eine Funktion *error()*, die ermittelt um *wieviel Prozent* der echte Preis vom vorhergesagten Preis abweicht - gemittelt über alle Autos.
Ihre Funktion erhält als Parameter die prognostizierten Preise und die echten Preise. 
Rufen Sie error() für Ihr Modell auf. Um wieviel Prozent weicht die kilometerstandsbasierte Prognose im Schnitt vom echten Preis ab?

In [5]:
def error(ypredicted, yreal):
    return (ypredicted / yreal) * 100

errorAvg = 0

for i in range(len(a)):
    calculatedPrice = calculatePrice(res[0], res[1], a[i])
    errorAvg += error(calculatedPrice, a[i])
    
errorAvg = errorAvg / len(a)
print(errorAvg)

## 6. Merkmale vergleichen
Versuchen Sie nun, den Preis nicht nur anhand des Kilometerstandes vorherzusagen, sondern versuchen Sie es mit *jedem* Merkmal (dem Zulassungsjahr, den PS, dem Kilometerstand, etc.). Für jedes Merkmal:
* Bestimmen Sie die Parameter a,b, mit leastsquares()
* Ermitteln Sie die vorhergesagten Preise
* Geben Sie den prozentualen Fehler aus, mit error().

Welches Merkmal ist das beste, d.h. es führt zum geringsten Fehler?

In [6]:
errorsList = []

for i in range(1, 57):
    ab = leastsquares(data[:, 0], data[:, i])
    calculatedPrice = 0
    
    for f in range(0, len(a)):
        calculatedPrice = calculatePrice(ab[0], ab[1], data[:, 0][f])
        errorsList.append(error(data[:, 0][f], calculatedPrice))
      
arr = np.asarray(errorsList) ** 2
argminIndex = arr.argmin();
print(argminIndex, " : ", arr[argminIndex] ** 0.5)